# convolution net model:
## layers i:
layers shape: [width, height, channel_i]


## kernels i:
+ convolution:
    kernels shape: [width, height, channel_(i-1), channel_i]
    forward: Z^[i] = W^[i] * Z^[i-1], 
    backward: dW^[i] += dZ^[i] * Z^[i-1], dZ^[i-1] += W^[i] * dZ^[i]
    
+ pooling:
    max        
    mean
             
+ full connected layers:
    


+ input layer 0: [width, height, 3(rgb)]

+ convolution output layer:

## Programming Parameters
+ kernels: 
    convolution: (layers, kernels, kernel_width, kernel_height, channel_i-1)
    pooling: max, mean
+ padding
+ step
+ layers: (layers, layer_width, layer_height, channel)
+ 





In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchvision
from PIL import Image

In [2]:
train_loader = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST('./../data/', train=True, download=False,
                                   transform=torchvision.transforms.Compose([
                                       torchvision.transforms.ToTensor(),
                                       torchvision.transforms.Normalize(
                                           (0.1307,), (0.3081,))
                                   ])),
        batch_size=128, shuffle=True)
def getXY():
    train_batch = enumerate(train_loader)
    batch_idx, (train_imgs, train_labels) = next(train_batch)
    return np.array(train_imgs), np.array(train_labels)

In [3]:
x, y = getXY()
img_array = x
print(img_array.shape)
print(y.shape)

(128, 1, 28, 28)
(128,)


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
img = Image.fromarray(img_array[2,0,:,:]*255)
img.show()

In [31]:
def get_W_b(n_c, n_c_pre, f_h, f_w):
    W = np.random.rand(n_c, n_c_pre, f_h, f_w)
    b = np.random.rand(n_c, 1, 1, 1)
    return W, b

In [6]:
def relu(Z):
    A = Z.copy()
    A[A < 0] = 0
    return A

def relu_(Z, dA):
    dZ = dA.copy()
    dZ[Z<0] = 0
    return dZ

softmax_layer_sum = 0
def softmax(Z):
    Z = Z - Z.max()
    A = np.exp(Z)
    softmax_layer_sum = A.sum()
    return A

def softmax_(Z):
    ex = np.exp(Z)
    return ex*(softmax_layer_sum-ex)/softmax_layer_sum**2

In [8]:
def get_mask(slice, mode):
    if mode=="max":
        mask = x == np.max(x)
    elif mode=="mean":
        mask = np.ones(slice.shape)
        mask = mask/mask.sum()
    return mask

In [13]:
def conv_forward(A_prev, W, b, stride, pad, mode="conv"):
    """
    Implements the forward propagation for a convolution function

    Arguments:
    A_prev -- output activations of the previous layer, numpy array of shape (m, n_C_prev, n_H_prev, n_W_prev)
    W -- Weights, numpy array of shape (n_C, n_C_prev, f, f)
    b -- Biases, numpy array of shape (n_C, 1, 1, 1)
    stride --
    pad --

    Returns:
    Z -- conv output, numpy array of shape (m, n_C, n_H, n_W)
    cache -- cache of values needed for the conv_backward() function
    """

    # get shape
    (m, n_C_prev, n_H_prev, n_W_prev) = A_prev.shape
    (n_C, n_C_prev, f, f) = W.shape

    # create container Z and padded A_prev
    n_H = int(np.floor((n_H_prev + 2 * pad - f) / stride + 1))
    n_W = int(np.floor((n_W_prev + 2 * pad - f) / stride + 1))
    Z = np.zeros((m, n_C, n_H, n_W))
    A = np.zeros((m, n_C, n_H, n_W))
    A_prev = np.pad(A_prev, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant', constant_values=(0))

    # propagation
    for i in range(0, m):
        for j in range(0, n_C):
            for k in range(0, n_H):
                for l in range(0, n_W):
                    assert (k*stride+f-1 < n_H_prev+2*pad and l*stride+f-1 <= n_W_prev+2*pad)
                    A_pre_slice = A_prev[i, :, k*stride:k*stride+f, l*stride:l*stride+f]
                    conv = []
                    if mode == "conv":
                        conv = A_pre_slice * W[j] + b[j]
                    elif mode == "max" or mode == "mean":
                        mask = get_mask(A_pre_slice, mode)
                        conv = A_pre_slice * mask
                    Z[i, j, k, l] += conv.sum()

    assert (Z.shape == (m, n_C, n_H, n_W))
    A = relu(Z)
    return Z, A

In [ ]:
def conv_backward(dA, Z, W, b, A_prev, stride, pad):
    
    # get shape
    (m, n_C_prev, n_H_prev, n_W_prev) = A_prev.shape
    (n_C, n_C_prev, f, f) = W.shape
    (m, n_C, n_H, n_W) = Z.shape
    
    # set gradient container
    dZ = relu_(Z, dA)
    dW = np.zeros((n_C, n_C_prev, f, f))
    db = np.zeros((n_C, 1, 1, 1))
    A_prev = np.pad(A_prev, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant', constant_values=(0))
    dA_prev = np.zeros(A_prev.shape)
    
    # propagation
    for i in range(0, m):
        for j in range(0, n_C):
            for k in range(0, n_H):
                for l in range(0, n_W):
                    assert (k*stride+f-1 < n_H_prev+2*pad and l*stride+f-1 <= n_W_prev+2*pad)
                    A_pre_slice = A_prev[i, :, k*stride:k*stride+f, l*stride:l*stride+f]
                    dW, db, dA_prev = [],[],[]
                    if mode == "conv":
                        dW[j] += dZ[i, j, k, l] * A_pre_slice
                        db[j] += dZ[i, j, k, l]
                        dA_prev[i, :, k*stride:k*stride+f, l*stride:l*stride+f] += W[j] * dZ[i, j, k, l]
                    elif mode == "max" or mode == "mean":
                        mask = get_mask(A_pre_slice, mode)
                        dA_prev[i, :, k*stride:k*stride+f, l*stride:l*stride+f] += mask * dZ[i, j, k, l]  
    
    return dW, db, dA_prev[:, :, pad:pad+n_H_prev, pad:pad+n_W_prev]

In [10]:
def fc_forward(A_prev, W, b, g):
    Z = np.dot(W, A_prec) + b
    A = g(Z)
    return Z, A 

In [11]:
def fc_backward(A_prev, Z, g_, dA):
    m, n_l, n_l_prev= Z.shape
    
    dZ = g_(Z)*dA
    dW = np.dot(dZ, A_prev.T)/m
    db = dA.sum(axis=1, keepdim=True)/m
    dA_prev = np.dot(W.T, dZ)
    
    return dW, db, dA_prev

In [66]:
def cross_lose(A, y):
    assert(A.shape == y.shape)
    return -y*np.log(A)
def cross_lose_(A, y):
    return -1/A*y

In [68]:
a = np.array([0.1,0.2,0.7])
y = np.array([0,0,1])
l = cross_lose(a, y)
l_ = cross_lose_(a, y)
print(l)
print(l_)
print(l*l_)

[-0.         -0.          0.35667494]
[-0.         -0.         -1.42857143]
[ 0.          0.         -0.50953563]


In [ ]:
def train(batchs, itrs, learning_rate):
    los = []
    
    # model data
    (A1, Z1, pad1, stride1, mode1, (W1, b1)) = ([], [], 1, 1, "conv", get_W_b(16, 1, 3, 3))
    (A2, Z2, pad2, stride2, mode2, (W2, b2)) = ([], [], 1, 2, "conv", get_W_b(16, 16, 3, 3))
    (A3, Z3, pad3, stride3, mode3, (W3, b3)) = ([], [], 1, 2, "max", get_W_b(0, 0, 0, 0))
    (A4, Z4, pad4, stride4, mode4, (W4, b4)) = ([], [], 0, 2, "conv", get_W_b(16, 16, 3, 3))
    (A5, Z5, pad5, stride5, mode5, (W5, b5)) = ([], [], 0, 1, "conv", get_W_b(32, 16, 3, 3))
    W6 = np.random.rand(16, 32)
    b6 = np.random.rand(16)
    W7 = np.random.rand(10, 16)
    b7 = np.random.rand(10)
    
    for i in range(0, batchs):
        A_0, y = getXY()
        for itr in range(0, itrs):
        # forward propagation
            # conv
            Z1, A1 = conv_forward(A_0, W1, b1, 1, 1, "conv")
            Z2, A2 = conv_forward(A_1, W2, b2, 1, 2, "conv")
            Z3, A3 = conv_forward(A_2, W3, b3, 1, 2, "max")
            Z4, A4 = conv_forward(A_3, W4, b4, 1, 2, "conv")
            Z5, A5 = conv_forward(A_4, W5, b5, 1, 1, "conv")
            
            # conv to fc
            m, A5_c, A5_h, A5_w = A5.shape 
            A5_ = A5.flatten('C')
            A5_ = A5_.reshape((m, -1), order='C').T
            
            # fc
            Z6, A6 = fc_forward(A5_, W6, b6, relu)
            Z7, A7 = fc_forward(A6, W7, b7, softmax)
            
        #lose
            l = cross_lose(A7, y)
            los.append(l)
            
        # backward propagation
            # fc
            dA7 = l * cross_lose_(A7, y)
            dW7, db7, dA6 = fc_backward(A6, Z7, softmax, dA7)
            dW6, db6, dA5_ = fc_backward(A5, Z6, relu, dA6)
            
            # fc to conv
            dA5 = dA5_.reshape(A5.shape, order='C')
            
            # conv
            dW5, db5 dA4 = conv_backward(dA5, Z5, W5, b5, A4, stride5, pad5, mode="conv")
            dW4, db4 dA3 = conv_backward(dA4, Z4, b4, A3, stride4, pad4, mode="conv")
            dW3, db3 dA2 = conv_backward(dA5, Z3, b3, A2, stride3, pad3, mode="max")
            dW2, db2 dA1 = conv_backward(dA5, Z3, b2, A1, stride2, pad2, mode="conv")
            dW1, db1 dA0 = conv_backward(dA5, Z1, b1, A0, stride1, pad1, mode="conv")
            
        # refresh the weights
            W7 = W7 - learning_rate*dW7
            b7 = b7 - learning_rate*b7
            W6 = W6 - learning_rate*dW6
            b6 = b6 - learning_rate*b6
            W5 = W5 - learning_rate*dW5
            b5 = b5 - learning_rate*b5
            W4 = W4 - learning_rate*dW4
            b4 = b4 - learning_rate*b4
            W3 = W3 - learning_rate*dW3
            b3 = b3 - learning_rate*b3
            W2 = W2 - learning_rate*dW2
            b2 = b2 - learning_rate*b2
            W1 = W1 - learning_rate*dW1
            b1 = b1 - learning_rate*b1
            
            
            
           
            
            
            


In [100]:
def get_mask(slice, mode):
    if mode=="max":
        mask = slice == np.max(slice)
    elif mode=="mean":
        mask = np.ones(slice.shape)
        mask = mask/np.sum(mask, axis=(1,2))
    return mask

In [123]:
a = np.array([
    [
        [1,2,3],
        [4,5,6],
        [7,8,0]
    ],
    [
        [1,2,3],
        [4,5,6],
        [7,8,9]
    ]
])
a.shape

(2, 3, 3)

In [117]:
b = np.sum(a, axis=(1,2), keepdims=True)
print(b)
print(b.shape)

[[[45]]

 [[45]]]
(2, 1, 1)


In [118]:
a/b

array([[[0.02222222, 0.04444444, 0.06666667],
        [0.08888889, 0.11111111, 0.13333333],
        [0.15555556, 0.17777778, 0.2       ]],

       [[0.02222222, 0.04444444, 0.06666667],
        [0.08888889, 0.11111111, 0.13333333],
        [0.15555556, 0.17777778, 0.2       ]]])

In [124]:
a == np.max(a, axis=(1,2),keepdims=True)

array([[[False, False, False],
        [False, False, False],
        [False,  True, False]],

       [[False, False, False],
        [False, False, False],
        [False, False,  True]]])